<a href="https://colab.research.google.com/github/RxnAch/DeepLearning/blob/main/Layers_and_Blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Layers and Blocks

To implement complex network,we introduce the concept of neural network,***block***.

A Block could describe a single layer, a component consisting of multiple layers, or the entire model itself.

From a programming standpoint, a block is represented by a class.


The following code generates a network with one fully-connected hidden layer with 256 units and ReLU activation, followed by a fully-connected output layer with 10 units(no activation function).

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20,256),
                    nn.ReLU(),
                    nn.Linear(256,10))
X = torch.rand(2,20)
net(X)

tensor([[ 0.1381, -0.1505, -0.2038, -0.0556, -0.0282, -0.0117,  0.0874, -0.1312,
         -0.0394, -0.0584],
        [ 0.0798, -0.1943, -0.1880,  0.0112, -0.0879,  0.0390,  0.1357, -0.0288,
         -0.0435, -0.0846]], grad_fn=<AddmmBackward>)

#A custom Block

Our block must possess:

1. Ingest input data as arguments to its forward propagation function.

2. Generate an output by having the forward propagation function return a value. Note that the output may have a different shape from the input. For example, the first fully-connected layer in our model above ingests an input of arbitrary dimension but returns an output of dimension 256.

3. Calculate the gradient of its output with respect to its input, which can be accessed via its backpropagation function. Typically this happens automatically.

4. Store and provide access to those parameters necessary to execute the forward propagation computation.

5. Initialize model parameters as needed.

In [31]:
class MLP(nn.Module): 
  #Declare a layer with model parameters.Here, we declare two fully connected layers
  def __init__(self):
    # Call the constructor of the `MLP` parent class `Module` to perform
    # the necessary initialization.
    super().__init__()
    self.hidden = nn.Linear(20,256) #hidden layer
    self.out = nn.Linear(256,10) #output layer
  #Define forward propagation with input "X"
  def forward(self,X):
    return self.out(F.relu(self.hidden(X)))



In [15]:
net = MLP()
net(X)

tensor([[-0.0944,  0.0666,  0.0340,  0.0122,  0.0773, -0.1533, -0.0289,  0.1357,
          0.0163, -0.2042],
        [-0.1267, -0.0008,  0.0189,  0.1058, -0.0131, -0.0216,  0.0414,  0.0085,
          0.1683, -0.3031]], grad_fn=<AddmmBackward>)

#My Sequential Block

To build our own simplified **MySequential**,
we just need to define two key functions:

1. A function to append blocks(module) one by one to a list.
2. Forward Propagation function in a order as given in list.

In [32]:
class MySequential(nn.Module):
  def __init__(self,*args):
    super().__init__()
    for idx,module in enumerate(args): #
      self._module[str(idx)] = module   #self.blocklist = [block for block in args]
  def forward(self,X):
    for block in self._modules.values():
      X = block(X)
    return X


In the __init__ method, we add every module to the ordered dictionary _modules one by one.

When our MySequential’s forward propagation function is invoked, each added block is executed in the order in which they were added. We can now reimplement an MLP using our MySequential class.

In [30]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0383, -0.1540, -0.1938, -0.1322, -0.1173,  0.1900, -0.0340,  0.1142,
          0.1248, -0.0362],
        [ 0.0403, -0.1120, -0.1187, -0.2979, -0.0204,  0.0447, -0.0974,  0.1629,
          0.0900,  0.1336]], grad_fn=<AddmmBackward>)

#Executing code in the forward propagation function

When greater flexibility is required, we will want to define our own blocks. For example, we might want to execute Python’s control flow within the forward propagation function. Moreover, we might want to perform arbitrary mathematical operations, not simply relying on predefined neural network layers.

In [22]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()
    #random weight parameters that will not compute gradients and thereafter remains constant
    self.rand_weight = torch.rand((20,20),requires_grad = False)
    self.linear = nn.Linear(20,20)
#Foraward Propagation function
#We can perform any mathematical operations. for now f(x,w) = c.w^T.x , c is constant which is not updated.
  def forward(self,X):
    X = self.linear(X)
    X = F.relu(torch.mm(X,self.rand_weight)+1)
    X = self.linear(X)
    #Control Flow
    while X.abs().sum()>1:
      X/=2
    return X.sum()

Note that this particular operation may not be useful in any real-world task. Our point is only to show you how to integrate arbitrary code into the flow of your neural network computations.



In [33]:
net = FixedHiddenMLP()
net(X)

tensor(0.1784, grad_fn=<SumBackward0>)

We can mix and match above various ways of assembling blocks together.

In [34]:
class NestMLP(nn.Module):#nest blocks
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),nn.Linear(64,32),nn.ReLU())#a block
    self.linear = nn.Linear(32,16) # a block
    #define forward propagation
  def forward(self,X):
    return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP()) #Sequential module chain blocks together.
chimera(X)

tensor(-0.3495, grad_fn=<SumBackward0>)